# Pinecone

Pinecone은 고성능 벡터 데이터베이스로, AI 및 머신러닝 애플리케이션을 위한 효율적인 벡터 저장 및 검색 솔루션입니다. 

Pinecone, Chroma, Faiss와 같은 벡터 데이터베이스들을 비교해보겠습니다. 

**Pinecone의 장점**

1. 확장성: 대규모 데이터셋에 대해 뛰어난 확장성을 제공합니다.
   
2. 관리 용이성: 완전 관리형 서비스로, 인프라 관리 부담이 적습니다.
   
3. 실시간 업데이트: 데이터의 실시간 삽입, 업데이트, 삭제가 가능합니다.
   
4. 고가용성: 클라우드 기반으로 높은 가용성과 내구성을 제공합니다.
   
5. API 친화적: RESTful/Python API를 통해 쉽게 통합할 수 있습니다.

**Pinecone의 단점**

1. 비용: Chroma나 Faiss에 비해 상대적으로 비용이 높을 수 있습니다.
   
2. 커스터마이징 제한: 완전 관리형 서비스이기 때문에 세부적인 커스터마이징에 제한이 있을 수 있습니다.
   
3. 데이터 위치: 클라우드에 데이터를 저장해야 하므로, 데이터 주권 문제가 있을 수 있습니다.

Chroma나 Faiss와 비교했을 때:

- Chroma/FAISS 오픈소스이며 로컬에서 실행 가능하여 초기 비용이 낮고 데이터 제어가 용이합니다. 커스터마이징의 자유도가 높습니다. 하지만 대규모 확장성 면에서는 Pinecone에 비해 제한적일 수 있습니다.

선택은 프로젝트의 규모, 요구사항, 예산 등을 고려하여 결정해야 합니다. 대규모 프로덕션 환경에서는 Pinecone이 유리할 수 있지만, 소규모 프로젝트나 실험적인 환경에서는 Chroma나 Faiss가 더 적합할 수 있습니다.

**참고**

- [Pinecone 공식 홈페이지](https://docs.pinecone.io/integrations/langchain)
- [Pinecone 랭체인](https://python.langchain.com/v0.2/docs/integrations/vectorstores/pinecone/)

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

## 한글 처리를 위한 불용어 사전
한글 불용어 사전 가져오기 (추후 토크나이저에 사용)

In [3]:
from langchain_teddynote.korean import stopwords

# 한글 불용어 사전 불러오기 (불용어 사전 출처: https://www.ranks.nl/stopwords/korean)
stopword = stopwords()
stopword[:20]

['아',
 '휴',
 '아이구',
 '아이쿠',
 '아이고',
 '어',
 '나',
 '우리',
 '저희',
 '따라',
 '의해',
 '을',
 '를',
 '에',
 '의',
 '가',
 '으로',
 '로',
 '에게',
 '뿐이다']

## 데이터 전처리

In [5]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import glob

# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)

split_docs = []

# 텍스트 파일을 load -> List[Document] 형태로 변환
files = sorted(glob.glob("data/*.pdf"))

for file in files:
    loader = PyMuPDFLoader(file)
    split_docs.extend(loader.load_and_split(text_splitter))

# 문서 개수 확인
len(split_docs)

4

In [6]:
split_docs[0].page_content

'<ADsP 요약정리 및 오답노트>\n-1과목-\n(객관식)\n데이터 마스킹 : 데이터의 속성은 유지한 채, 익명으로 생성\nCinematch -> 넷플릭스에서 개발한 알고리즘\n데이터마이닝 vs 머신러닝(딥러닝) 구분하기 다른거임\n트레이딩, 공급, 수요예측 -> 에너지 산업\nCRM -> 고객관계관리 데이터베이스 (기업내부)\nERP -> 기업 전체를 통합적으로 관리하고 경영의 효율화 목적\n빅데이터 가치측정 어려윤 이유 : 1) 데이터 재사용,재조합,다목적용 개발'

Pinecone 에 DB 저장하기 위한 문서 전처리를 수행합니다. 이 과정에서 `metadata_keys` 를 지정할 수 있습니다.

추가로 metadata 를 태깅하고 싶은 경우 사전 처리 작업에서 미리 metadata 를 추가한 뒤 진행합니다.

- `split_docs`: 문서 분할 결과를 담은 List[Document] 입니다.
- `metadata_keys`: 문서에 추가할 metadata 키를 담은 List 입니다.
- `min_length`: 문서의 최소 길이를 지정합니다. 이 길이보다 짧은 문서는 제외합니다.
- `use_basename`: 소스 경로를 기준으로 파일명을 사용할지 여부를 지정합니다. 기본값은 `False` 입니다.

In [7]:
# metadata 를 확인합니다.
split_docs[0].metadata

{'source': 'data\\ADsP.pdf',
 'file_path': 'data\\ADsP.pdf',
 'page': 0,
 'total_pages': 1,
 'format': 'PDF 1.4',
 'title': '',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': '',
 'producer': 'PyPDF2',
 'creationDate': '',
 'modDate': '',
 'trapped': ''}

### 문서의 전처리

- 필요한 `metadata` 정보를 추출합니다.
- 최소 길이 이상의 데이만 필터링 합니다.
  
- 문서의 `basename` 을 사용할지 여부를 지정합니다. 기본값은 `False` 입니다.
  - 여기서 `basename` 이란 파일 경로의 가장 마지막 부분을 의미합니다. 
  - 예를 들어, `/Users/teddy/data/document.pdf` 의 경우 `document.pdf` 가 됩니다.

In [8]:
split_docs[0].metadata

{'source': 'data\\ADsP.pdf',
 'file_path': 'data\\ADsP.pdf',
 'page': 0,
 'total_pages': 1,
 'format': 'PDF 1.4',
 'title': '',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': '',
 'producer': 'PyPDF2',
 'creationDate': '',
 'modDate': '',
 'trapped': ''}

In [9]:
split_docs[0].page_content

'<ADsP 요약정리 및 오답노트>\n-1과목-\n(객관식)\n데이터 마스킹 : 데이터의 속성은 유지한 채, 익명으로 생성\nCinematch -> 넷플릭스에서 개발한 알고리즘\n데이터마이닝 vs 머신러닝(딥러닝) 구분하기 다른거임\n트레이딩, 공급, 수요예측 -> 에너지 산업\nCRM -> 고객관계관리 데이터베이스 (기업내부)\nERP -> 기업 전체를 통합적으로 관리하고 경영의 효율화 목적\n빅데이터 가치측정 어려윤 이유 : 1) 데이터 재사용,재조합,다목적용 개발'

In [12]:
from langchain_teddynote.community.pinecone import preprocess_documents

contents, metadatas = preprocess_documents(
    split_docs=split_docs,
    metadata_keys=["source", "page", "author"],
    min_length=5,
    use_basename=True,
)

  0%|          | 0/4 [00:00<?, ?it/s]

In [13]:
metadatas["source"][:5]

['ADsP.pdf', 'ADsP.pdf', 'ADsP.pdf', 'ADsP.pdf']

In [14]:
contents[:5]

['<ADsP 요약정리 및 오답노트>\n-1과목-\n(객관식)\n데이터 마스킹 : 데이터의 속성은 유지한 채, 익명으로 생성\nCinematch -> 넷플릭스에서 개발한 알고리즘\n데이터마이닝 vs 머신러닝(딥러닝) 구분하기 다른거임\n트레이딩, 공급, 수요예측 -> 에너지 산업\nCRM -> 고객관계관리 데이터베이스 (기업내부)\nERP -> 기업 전체를 통합적으로 관리하고 경영의 효율화 목적\n빅데이터 가치측정 어려윤 이유 : 1) 데이터 재사용,재조합,다목적용 개발',
 '빅데이터 가치측정 어려윤 이유 : 1) 데이터 재사용,재조합,다목적용 개발\n                                          2) 새로운 가치 창출\n                                          3) 분석 기술 발전\ncf. 전문인력증가는 가치측정과 관련 없음\n사생활 침해를 막기 위한 개인정보 무작위 처리 (본래 목적 외에 사용 방지기술) -> 난수화\n유형분석 -> 특성에따라 분류할때 사용한다.\n핀테크(금융)분야에서 빅데이터 활용의 핵심분야 -> 신용평가',
 '핀테크(금융)분야에서 빅데이터 활용의 핵심분야 -> 신용평가\nK-NN 분석기법 - 딥러닝과 관련 x\nLSTM, Autoencoder, RNN -> 딥러닝과 관련 o\nCaffe, Tnsorflow, Theano -> 딥러닝 관련된 오픈소스\nAnaconda -> 머신러닝 관련된 오픈소스\n책임 원칙의 훼손 -> 일어나지도 않은 일을 예측해서 행동함 (범행 전에 체포, 신용불량 전에 \n대출금지)\n멀티미디어 등 복잡한 데이터베이스 관리 -> 객체지향 DBMS\ncf. 일반적으로 사용되는 테이블 기반 -> 관계형 DBMS',
 'cf. 일반적으로 사용되는 테이블 기반 -> 관계형 DBMS\n데이터 시각화 -> 비즈니스 컨설팅 영역 (IT영역 X)']

In [15]:
# VectorStore 에 저장할 metadata 확인
metadatas.keys()

dict_keys(['source', 'page', 'author'])

In [16]:
# metadata 에서 source 를 확인합니다.
metadatas["source"][:5]

['ADsP.pdf', 'ADsP.pdf', 'ADsP.pdf', 'ADsP.pdf']

In [17]:
# 문서 개수 확인, 소스 개수 확인, 페이지 개수 확인
len(contents), len(metadatas["source"]), len(metadatas["page"])

(4, 4, 4)

### API 키 발급

- [링크](https://app.pinecone.io/)
- 프로필 - Account - Projects - Starter - API keys - 발급

`.env` 파일에 아래와 같이 추가합니다.

```
PINECONE_API_KEY="YOUR_PINECONE_API_KEY"
```

## 새로운 VectorStore 인덱스 생성

Pinecone 의 새로운 인덱스를 생성합니다.
pinecone 인덱스를 생성합니다.

**주의사항**
- `metric` 은 유사도 측정 방법을 지정합니다. 만약 HybridSearch 를 고려하고 있다면 `metric` 은 `dotproduct` 로 지정합니다.

In [20]:
import os
from langchain_teddynote.community.pinecone import create_index

# Pinecone 인덱스 생성
pc_index = create_index(
    api_key=os.environ["PINECONE_API_KEY"],
    index_name="db-index",  # 인덱스 이름을 지정합니다.
    dimension=4096,  # Embedding 차원과 맞춥니다. (OpenAIEmbeddings: 1536, UpstageEmbeddings: 4096)
    metric="dotproduct",  # 유사도 측정 방법을 지정합니다. (dotproduct, euclidean, cosine)
)

[create_index]
{'dimension': 4096,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 0}},
 'total_vector_count': 0}


아래는 **유료 Pod** 를 사용하는 예시입니다. **유료 Pod** 는 무료 Serverless Pod 대비 더 확장된 기능을 제공합니다.

- 참고: https://docs.pinecone.io/guides/indexes/choose-a-pod-type-and-size

In [ ]:
import os
from langchain_teddynote.community.pinecone import create_index
from pinecone import PodSpec

# Pinecone 인덱스 생성
pc_index = create_index(
    api_key=os.environ["PINECONE_API_KEY"],
    index_name="teddynote-db-index2",  # 인덱스 이름을 지정합니다.
    dimension=4096,  # Embedding 차원과 맞춥니다. (OpenAIEmbeddings: 1536, UpstageEmbeddings: 4096)
    metric="dotproduct",  # 유사도 측정 방법을 지정합니다. (dotproduct, euclidean, cosine)
    pod_spec=PodSpec(
        environment="us-west1-gcp", pod_type="p1.x1", pods=1
    ),  # 유료 Pod 사용
)

## Sparse Encoder 생성

- Sparse Encoder 를 생성합니다. 
- `Kiwi Tokenizer` 와 한글 불용어(stopwords) 처리를 수행합니다.
- Sparse Encoder 를 활용하여 contents 를 학습합니다. 여기서 학습한 인코드는 VectorStore 에 문서를 저장할 때 Sparse Vector 를 생성할 때 활용합니다.

In [21]:
from langchain_teddynote.community.pinecone import (
    create_sparse_encoder,
    fit_sparse_encoder,
)

# 한글 불용어 사전 + Kiwi 형태소 분석기를 사용합니다.
sparse_encoder = create_sparse_encoder(stopwords(), mode="kiwi")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cava2\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


Sparse Encoder 에 Corpus 를 학습합니다.

- `save_path`: Sparse Encoder 를 저장할 경로입니다. 추후에 `pickle` 형식으로 저장한 Sparse Encoder 를 불러와 Query 임베딩할 때 사용합니다. 따라서, 이를 저장할 경로를 지정합니다.

In [22]:
# Sparse Encoder 를 사용하여 contents 를 학습
saved_path = fit_sparse_encoder(
    sparse_encoder=sparse_encoder, contents=contents, save_path="./sparse_encoder.pkl"
)

  0%|          | 0/4 [00:00<?, ?it/s]

[fit_sparse_encoder]
Saved Sparse Encoder to: ./sparse_encoder.pkl


### Pinecone: DB Index에 추가 (Upsert)
- `context`: 문서의 내용입니다.
- `page`: 문서의 페이지 번호입니다.
- `source`: 문서의 출처입니다.
- `values`: Embedder 를 통해 얻은 문서의 임베딩입니다.
- `sparse values`: Sparse Encoder 를 통해 얻은 문서의 임베딩입니다.

In [23]:
from langchain_openai import OpenAIEmbeddings
from langchain_upstage import UpstageEmbeddings

# 임베딩 모델 생성
openai_embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
upstage_embeddings = UpstageEmbeddings(model="solar-embedding-1-large-passage")

In [35]:
%%time
from langchain_teddynote.community.pinecone import upsert_documents
from langchain_upstage import UpstageEmbeddings

upsert_documents(
    index=pc_index,  # Pinecone 인덱스
    namespace="teddynote-namespace-01",  # Pinecone namespace
    contents=contents,  # 이전에 전처리한 문서 내용
    metadatas=metadatas,  # 이전에 전처리한 문서 메타데이터
    sparse_encoder=sparse_encoder,  # Sparse encoder
    embedder=upstage_embeddings,
    batch_size=32,
)

  0%|          | 0/1 [00:00<?, ?it/s]

[upsert_documents]
{'dimension': 4096,
 'index_fullness': 0.0,
 'namespaces': {'teddynote-namespace-01': {'vector_count': 0}},
 'total_vector_count': 0}
CPU times: total: 93.8 ms
Wall time: 2.75 s


아래는 분산처리를 수행하여 대용량 문서를 빠르게 Upsert 합니다. 대용량 업로드시 활용하세요.

In [ ]:
%%time
from langchain_teddynote.community.pinecone import upsert_documents_parallel

upsert_documents_parallel(
    index=pc_index,  # Pinecone 인덱스
    namespace="teddynote-namespace-02",  # Pinecone namespace
    contents=contents,  # 이전에 전처리한 문서 내용
    metadatas=metadatas,  # 이전에 전처리한 문서 메타데이터
    sparse_encoder=sparse_encoder,  # Sparse encoder
    embedder=upstage_embeddings,
    batch_size=64,
    max_workers=30,
)

## 인덱스 조회/삭제

`describe_index_stats` 메서드는 인덱스의 내용에 대한 통계 정보를 제공합니다. 이 메서드를 통해 네임스페이스별 벡터 수와 차원 수 등의 정보를 얻을 수 있습니다.

**매개변수**
* `filter` (Optional[Dict[str, Union[str, float, int, bool, List, dict]]]): 특정 조건에 맞는 벡터들에 대한 통계만 반환하도록 하는 필터. 기본값은 None
* `**kwargs`: 추가 키워드 인자

**반환값**
* `DescribeIndexStatsResponse`: 인덱스에 대한 통계 정보를 담고 있는 객체

**사용 예시**
* 기본 사용: `index.describe_index_stats()`
* 필터 적용: `index.describe_index_stats(filter={'key': 'value'})`

**참고**
- metadata 필터링은 유료 사용자에 한하여 가능합니다.

In [25]:
# 인덱스 조회
pc_index.describe_index_stats()

{'dimension': 4096,
 'index_fullness': 0.0,
 'namespaces': {'teddynote-namespace-01': {'vector_count': 4}},
 'total_vector_count': 4}

In [26]:
from langchain_teddynote.community.pinecone import delete_namespace

delete_namespace(
    pinecone_index=pc_index,
    namespace="teddynote-namespace-01",
)

네임스페이스 'teddynote-namespace-01'의 모든 데이터가 삭제되었습니다.


In [27]:
pc_index.describe_index_stats()

{'dimension': 4096,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 0}},
 'total_vector_count': 0}

## 검색기(Retriever) 생성

### PineconeKiwiHybridRetriever 초기화 파라미터 설정

`init_pinecone_index` 함수와 `PineconeKiwiHybridRetriever` 클래스는 Pinecone을 사용한 하이브리드 검색 시스템을 구현합니다. 이 시스템은 밀집 벡터와 희소 벡터를 결합하여 효과적인 문서 검색을 수행합니다.

**Pinecone 인덱스 초기화**

`init_pinecone_index` 함수는 Pinecone 인덱스를 초기화하고 필요한 구성 요소를 설정합니다.

**매개변수**
* `index_name` (str): Pinecone 인덱스 이름
* `namespace` (str): 사용할 네임스페이스
* `api_key` (str): Pinecone API 키
* `sparse_encoder_pkl_path` (str): 희소 인코더 피클 파일 경로
* `stopwords` (List[str]): 불용어 리스트
* `tokenizer` (str): 사용할 토크나이저 (기본값: "kiwi")
* `embeddings` (Embeddings): 임베딩 모델
* `top_k` (int): 반환할 최대 문서 수 (기본값: 10)
* `alpha` (float): 밀집 벡터와 희소 벡터의 가중치 조절 파라미터 (기본값: 0.5)

**주요 기능**
1. Pinecone 인덱스 초기화 및 통계 정보 출력
2. 희소 인코더(BM25) 로딩 및 토크나이저 설정
3. 네임스페이스 지정

In [36]:
import os
from langchain_teddynote.korean import stopwords
from langchain_teddynote.community.pinecone import init_pinecone_index
from langchain_upstage import UpstageEmbeddings

pinecone_params = init_pinecone_index(
    index_name="db-index",  # Pinecone 인덱스 이름
    namespace="teddynote-namespace-01",  # Pinecone Namespace
    api_key=os.environ["PINECONE_API_KEY"],  # Pinecone API Key
    sparse_encoder_path="./sparse_encoder.pkl",  # Sparse Encoder 저장경로(save_path)
    stopwords=stopwords(),  # 불용어 사전
    tokenizer="kiwi",
    embeddings=UpstageEmbeddings(
        model="solar-embedding-1-large-query"
    ),  # Dense Embedder
    top_k=5,  # Top-K 문서 반환 개수
    alpha=0.5,  # alpha=0.75로 설정한 경우, (0.75: Dense Embedding, 0.25: Sparse Embedding)
)

[init_pinecone_index]
{'dimension': 4096,
 'index_fullness': 0.0,
 'namespaces': {'teddynote-namespace-01': {'vector_count': 0}},
 'total_vector_count': 0}


### PineconeKiwiHybridRetriever

`PineconeKiwiHybridRetriever` 클래스는 Pinecone과 Kiwi를 결합한 하이브리드 검색기를 구현합니다.

**주요 속성**
* `embeddings`: 밀집 벡터 변환용 임베딩 모델
* `sparse_encoder`: 희소 벡터 변환용 인코더
* `index`: Pinecone 인덱스 객체
* `top_k`: 반환할 최대 문서 수
* `alpha`: 밀집 벡터와 희소 벡터의 가중치 조절 파라미터
* `namespace`: Pinecone 인덱스 내 네임스페이스

**특징**
* 밀집 벡터와 희소 벡터를 결합한 HybridSearch Retriever
* 가중치 조절을 통한 검색 전략 최적화 가능
* 다양한 동적 metadata 필터링 적용 가능(`search_kwargs` 사용: `filter`, `k`, `rerank`, `rerank_model`, `top_n` 등)

**사용 예시**
1. `init_pinecone_index` 함수로 필요한 구성 요소 초기화
2. 초기화된 구성 요소로 `PineconeKiwiHybridRetriever` 인스턴스 생성
3. 생성된 검색기를 사용하여 하이브리드 검색 수행

In [37]:
from langchain_teddynote.community.pinecone import PineconeKiwiHybridRetriever

# 검색기 생성
pinecone_retriever = PineconeKiwiHybridRetriever(**pinecone_params)

In [38]:
# 실행 결과
search_results = pinecone_retriever.invoke("데이터 마스킹 관련 정보에 대해서 알려줘")
for result in search_results:
    print(result.page_content)
    print(result.metadata)
    print("\n====================\n")

<ADsP 요약정리 및 오답노트>
-1과목-
(객관식)
데이터 마스킹 : 데이터의 속성은 유지한 채, 익명으로 생성
Cinematch -> 넷플릭스에서 개발한 알고리즘
데이터마이닝 vs 머신러닝(딥러닝) 구분하기 다른거임
트레이딩, 공급, 수요예측 -> 에너지 산업
CRM -> 고객관계관리 데이터베이스 (기업내부)
ERP -> 기업 전체를 통합적으로 관리하고 경영의 효율화 목적
빅데이터 가치측정 어려윤 이유 : 1) 데이터 재사용,재조합,다목적용 개발
{'author': '', 'context': '<ADsP 요약정리 및 오답노트>\n-1과목-\n(객관식)\n데이터 마스킹 : 데이터의 속성은 유지한 채, 익명으로 생성\nCinematch -> 넷플릭스에서 개발한 알고리즘\n데이터마이닝 vs 머신러닝(딥러닝) 구분하기 다른거임\n트레이딩, 공급, 수요예측 -> 에너지 산업\nCRM -> 고객관계관리 데이터베이스 (기업내부)\nERP -> 기업 전체를 통합적으로 관리하고 경영의 효율화 목적\n빅데이터 가치측정 어려윤 이유 : 1) 데이터 재사용,재조합,다목적용 개발', 'page': 0.0, 'source': 'ADsP.pdf'}


빅데이터 가치측정 어려윤 이유 : 1) 데이터 재사용,재조합,다목적용 개발
                                          2) 새로운 가치 창출
                                          3) 분석 기술 발전
cf. 전문인력증가는 가치측정과 관련 없음
사생활 침해를 막기 위한 개인정보 무작위 처리 (본래 목적 외에 사용 방지기술) -> 난수화
유형분석 -> 특성에따라 분류할때 사용한다.
핀테크(금융)분야에서 빅데이터 활용의 핵심분야 -> 신용평가
{'author': '', 'context': '빅데이터 가치측정 어려윤 이유 : 1) 데이터 재사용,재조합,다목적용 개발\n                                          2) 새로운 가치 창출\n   

동적 `search_kwargs` 사용
- `k`: 반환할 최대 문서 수 지정

In [39]:
# 실행 결과
search_results = pinecone_retriever.invoke(
    "데이터 관련 정보에 대해서 알려줘", search_kwargs={"k": 1}
)
for result in search_results:
    print(result.page_content)
    print(result.metadata)
    print("\n====================\n")

빅데이터 가치측정 어려윤 이유 : 1) 데이터 재사용,재조합,다목적용 개발
                                          2) 새로운 가치 창출
                                          3) 분석 기술 발전
cf. 전문인력증가는 가치측정과 관련 없음
사생활 침해를 막기 위한 개인정보 무작위 처리 (본래 목적 외에 사용 방지기술) -> 난수화
유형분석 -> 특성에따라 분류할때 사용한다.
핀테크(금융)분야에서 빅데이터 활용의 핵심분야 -> 신용평가
{'author': '', 'context': '빅데이터 가치측정 어려윤 이유 : 1) 데이터 재사용,재조합,다목적용 개발\n                                          2) 새로운 가치 창출\n                                          3) 분석 기술 발전\ncf. 전문인력증가는 가치측정과 관련 없음\n사생활 침해를 막기 위한 개인정보 무작위 처리 (본래 목적 외에 사용 방지기술) -> 난수화\n유형분석 -> 특성에따라 분류할때 사용한다.\n핀테크(금융)분야에서 빅데이터 활용의 핵심분야 -> 신용평가', 'page': 0.0, 'source': 'ADsP.pdf'}




동적 `search_kwargs` 사용
- `alpha`: 밀집 벡터와 희소 벡터의 가중치 조절 파라미터. 0과 1 사이의 값을 지정합니다. `0.5` 가 기본값이고, 1에 가까울수록 dense 벡터의 가중치가 높아집니다.

In [41]:
# 실행 결과
search_results = pinecone_retriever.invoke(
    "데이터마스킹", search_kwargs={"alpha": 1, "k": 1}
)
for result in search_results:
    print(result.page_content)
    print(result.metadata)
    print("\n====================\n")

<ADsP 요약정리 및 오답노트>
-1과목-
(객관식)
데이터 마스킹 : 데이터의 속성은 유지한 채, 익명으로 생성
Cinematch -> 넷플릭스에서 개발한 알고리즘
데이터마이닝 vs 머신러닝(딥러닝) 구분하기 다른거임
트레이딩, 공급, 수요예측 -> 에너지 산업
CRM -> 고객관계관리 데이터베이스 (기업내부)
ERP -> 기업 전체를 통합적으로 관리하고 경영의 효율화 목적
빅데이터 가치측정 어려윤 이유 : 1) 데이터 재사용,재조합,다목적용 개발
{'author': '', 'context': '<ADsP 요약정리 및 오답노트>\n-1과목-\n(객관식)\n데이터 마스킹 : 데이터의 속성은 유지한 채, 익명으로 생성\nCinematch -> 넷플릭스에서 개발한 알고리즘\n데이터마이닝 vs 머신러닝(딥러닝) 구분하기 다른거임\n트레이딩, 공급, 수요예측 -> 에너지 산업\nCRM -> 고객관계관리 데이터베이스 (기업내부)\nERP -> 기업 전체를 통합적으로 관리하고 경영의 효율화 목적\n빅데이터 가치측정 어려윤 이유 : 1) 데이터 재사용,재조합,다목적용 개발', 'page': 0.0, 'source': 'ADsP.pdf'}




In [42]:
# 실행 결과
search_results = pinecone_retriever.invoke(
    "데이터마스킹", search_kwargs={"alpha": 1, "k": 1}
)
for result in search_results:
    print(result.page_content)
    print(result.metadata)
    print("\n====================\n")

<ADsP 요약정리 및 오답노트>
-1과목-
(객관식)
데이터 마스킹 : 데이터의 속성은 유지한 채, 익명으로 생성
Cinematch -> 넷플릭스에서 개발한 알고리즘
데이터마이닝 vs 머신러닝(딥러닝) 구분하기 다른거임
트레이딩, 공급, 수요예측 -> 에너지 산업
CRM -> 고객관계관리 데이터베이스 (기업내부)
ERP -> 기업 전체를 통합적으로 관리하고 경영의 효율화 목적
빅데이터 가치측정 어려윤 이유 : 1) 데이터 재사용,재조합,다목적용 개발
{'author': '', 'context': '<ADsP 요약정리 및 오답노트>\n-1과목-\n(객관식)\n데이터 마스킹 : 데이터의 속성은 유지한 채, 익명으로 생성\nCinematch -> 넷플릭스에서 개발한 알고리즘\n데이터마이닝 vs 머신러닝(딥러닝) 구분하기 다른거임\n트레이딩, 공급, 수요예측 -> 에너지 산업\nCRM -> 고객관계관리 데이터베이스 (기업내부)\nERP -> 기업 전체를 통합적으로 관리하고 경영의 효율화 목적\n빅데이터 가치측정 어려윤 이유 : 1) 데이터 재사용,재조합,다목적용 개발', 'page': 0.0, 'source': 'ADsP.pdf'}




**Metadata 필터링**
동적 `search_kwargs` 사용
- `filter`: metadata 필터링 적용


(예시) `page` 가 5보다 작은 문서만 검색합니다.

In [43]:
# 실행 결과
search_results = pinecone_retriever.invoke(
    "앤스로픽의 claude 출시 관련 내용을 알려줘",
    search_kwargs={"filter": {"page": {"$lt": 5}}, "k": 2},
)
for result in search_results:
    print(result.page_content)
    print(result.metadata)
    print("\n====================\n")

핀테크(금융)분야에서 빅데이터 활용의 핵심분야 -> 신용평가
K-NN 분석기법 - 딥러닝과 관련 x
LSTM, Autoencoder, RNN -> 딥러닝과 관련 o
Caffe, Tnsorflow, Theano -> 딥러닝 관련된 오픈소스
Anaconda -> 머신러닝 관련된 오픈소스
책임 원칙의 훼손 -> 일어나지도 않은 일을 예측해서 행동함 (범행 전에 체포, 신용불량 전에 
대출금지)
멀티미디어 등 복잡한 데이터베이스 관리 -> 객체지향 DBMS
cf. 일반적으로 사용되는 테이블 기반 -> 관계형 DBMS
{'author': '', 'context': '핀테크(금융)분야에서 빅데이터 활용의 핵심분야 -> 신용평가\nK-NN 분석기법 - 딥러닝과 관련 x\nLSTM, Autoencoder, RNN -> 딥러닝과 관련 o\nCaffe, Tnsorflow, Theano -> 딥러닝 관련된 오픈소스\nAnaconda -> 머신러닝 관련된 오픈소스\n책임 원칙의 훼손 -> 일어나지도 않은 일을 예측해서 행동함 (범행 전에 체포, 신용불량 전에 \n대출금지)\n멀티미디어 등 복잡한 데이터베이스 관리 -> 객체지향 DBMS\ncf. 일반적으로 사용되는 테이블 기반 -> 관계형 DBMS', 'page': 0.0, 'source': 'ADsP.pdf'}


빅데이터 가치측정 어려윤 이유 : 1) 데이터 재사용,재조합,다목적용 개발
                                          2) 새로운 가치 창출
                                          3) 분석 기술 발전
cf. 전문인력증가는 가치측정과 관련 없음
사생활 침해를 막기 위한 개인정보 무작위 처리 (본래 목적 외에 사용 방지기술) -> 난수화
유형분석 -> 특성에따라 분류할때 사용한다.
핀테크(금융)분야에서 빅데이터 활용의 핵심분야 -> 신용평가
{'author': '', 'context': '빅데이터 가치측정 어려윤 이유 : 1) 데이터 재사용,재조합,다목적

동적 `search_kwargs` 사용
- `filter`: metadata 필터링 적용

(예시) `source` 가 `SPRi AI Brief_8월호_산업동향.pdf` 문서내 검색합니다.

In [44]:
# 실행 결과
search_results = pinecone_retriever.invoke(
    "데이터 마스킹 관련 내용을 알려줘",
    search_kwargs={
        "filter": {"source": {"$eq": "SPRi AI Brief_8월호_산업동향.pdf"}},
        "k": 3,
    },
)
for result in search_results:
    print(result.page_content)
    print(result.metadata)
    print("\n====================\n")

## Reranking 적용

- 동적 reranking 기능을 구현해 놓았지만, pinecone 라이브러리 의존성에 문제가 있을 수 있습니다.
- 따라서, 아래 코드는 향후 의존성 해결 후 원활하게 동작할 수 있습니다.

참고 문서: https://docs.pinecone.io/guides/inference/rerank

In [ ]:
# reranker 미사용
retrieval_results = pinecone_retriever.invoke(
    "앤스로픽의 클로드 소넷",
)

# BGE-reranker-v2-m3 모델 사용
reranked_results = pinecone_retriever.invoke(
    "앤스로픽의 클로드 소넷",
    search_kwargs={"rerank": True, "rerank_model": "bge-reranker-v2-m3", "top_n": 3},
)